In [1]:
import numpy as np

from itertools import product

In [2]:
# constructing the phase augmented w state

num_qubits = 20

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
w_aug = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    w_aug[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)


def format_bytes(b):
    for u in ['B', 'KB', 'MB', 'GB', 'TB']:
        if b < 1024:
            return f"{b:.2f} {u}"
        b /= 1024


print(f"Size of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

for i in range(10):
    print(f"{i:0{num_qubits}b}: {w_aug[i]:.2f} + {w_aug[i].imag:.2f}j")

Size of state vector in memory: 16.00 MB 

00000000000000000000: 0.00+0.00j + 0.00j
00000000000000000001: -0.15-0.16j + -0.16j
00000000000000000010: 0.10-0.20j + -0.20j
00000000000000000011: 0.00+0.00j + 0.00j
00000000000000000100: 0.21+0.09j + 0.09j
00000000000000000101: 0.00+0.00j + 0.00j
00000000000000000110: 0.00+0.00j + 0.00j
00000000000000000111: 0.00+0.00j + 0.00j
00000000000000001000: -0.21-0.08j + -0.08j
00000000000000001001: 0.00+0.00j + 0.00j


In [3]:
class PauliOperator:
    def __init__(self, matrix, eigenvalues, eigenvectors):
        """
        Bundles the relevant information of a Pauli operator into a single object.

        REMARK: The eigenvalues and eigenvectors are directly passed to avoid numerical inaccuracies.

        :param matrix: The matrix representation of the Pauli operator.
        :param eigenvalues: The eigenvalues of the Pauli operator.
        :param eigenvectors: The eigenvectors of the Pauli operator.
        """
        self.matrix = matrix
        self.eigenvalues = eigenvalues
        self.eigenvectors = eigenvectors


class MultiQubitMeasurementSetting:
    def __init__(self, measurement_settings):
        """
        Represents a multi-qubit measurement setting using Pauli operators.

        :param measurement_settings: A list of Pauli operators representing the measurement setting for each qubit.
        """
        self._measurement_settings = measurement_settings
        self.eigenvectors = []
        self.eigenvalues = []

        self._construct_basis()

    def _construct_basis(self):
        """
        Constructs the multi-qubit basis vectors and their corresponding eigenvalues from the given Pauli operators.
        """

        # in this first loop we iterate over the computational basis states of the n-qubit system
        # constructing them as a list of 0s and 1s allows us to leverage pythons addressing capabilities
        for state_addressing in product(range(2), repeat=len(self._measurement_settings)):
            multi_qubit_eigenvalue = 1.0
            multi_qubit_eigenvector = None

            # we pair up the Pauli operator for each single qubit basis state and retrieve the matching eigenvalue and eigenvector
            for pauli_op, state_index in zip(self._measurement_settings, state_addressing):
                eigenvalue = pauli_op.eigenvalues[state_index]
                eigenvector = pauli_op.eigenvectors[state_index]

                # the eigenvalue of the multi-qubit operator is the product of the single-qubit eigenvalues
                multi_qubit_eigenvalue *= eigenvalue

                # we build up the multi-qubit eigenvector by taking the tensor product of the single-qubit eigenvectors
                if multi_qubit_eigenvector is None:
                    multi_qubit_eigenvector = eigenvector
                else:
                    multi_qubit_eigenvector = np.kron(multi_qubit_eigenvector, eigenvector)

            self.eigenvalues.append(multi_qubit_eigenvalue)
            self.eigenvectors.append(multi_qubit_eigenvector)

In [4]:
sigma_0 = PauliOperator(np.eye(2), [1, 1], [np.array([1, 0]), np.array([0, 1])])
sigma_x = PauliOperator(np.array([[0, 1], [1, 0]]), [1, -1], [np.array([1, 1]) / np.sqrt(2), np.array([1, -1]) / np.sqrt(2)])
sigma_y = PauliOperator(np.array([[0, -1j], [1j, 0]]), [1, -1], [np.array([1, -1j]) / np.sqrt(2), np.array([1, 1j]) / np.sqrt(2)])
sigma_z = PauliOperator(np.array([[1, 0], [0, -1]]), [1, -1], [np.array([1, 0]), np.array([0, 1])])
pauli_operators = { 'I': sigma_0, 'X': sigma_x, 'Y': sigma_y, 'Z': sigma_z }

In [5]:
import numpy as np

def sample_state_fast(state, pauli_labels, *, rng=np.random.default_rng()):
    """
    Single-shot measurement of |state⟩ in the tensor-product basis defined by
    pauli_labels, without constructing the 4^n-sized eigenbasis.

    Parameters
    ----------
    state : np.ndarray, shape=(2**n,), complex
        Normalised state vector.  Will be *modified in place* (projection).
    pauli_labels : Sequence[str]
        One character per qubit: 'I', 'X', 'Y' or 'Z'.
    rng : np.random.Generator, optional
        RNG (defaults to NumPy’s PCG64).

    Returns
    -------
    str
        Measurement bit-string (‘0’ = +1 eigenvalue, ‘1’ = −1).
    """
    n = len(pauli_labels)
    psi = state.reshape((2,) * n)           # view |ψ⟩ as n-dimensional tensor
    outcome_bits = []
    sqrt2 = np.sqrt(2.0)

    for q, pl in enumerate(pauli_labels):
        if pl == 'I':
            outcome_bits.append('0')
            continue

        axis = n - 1 - q                    # axis 0 = MSB
        psi0 = np.take(psi, 0, axis=axis)   # slice with qubit q = |0⟩
        psi1 = np.take(psi, 1, axis=axis)   # slice with qubit q = |1⟩

        # --- probability p0 of getting eigenvalue +1 -----------------------
        if pl == 'Z':
            p0 = np.sum(np.abs(psi0) ** 2)
        elif pl == 'X':
            p0 = np.sum(np.abs((psi0 + psi1) / sqrt2) ** 2)
        elif pl == 'Y':
            p0 = np.sum(np.abs((psi0 + 1j * psi1) / sqrt2) ** 2)
        else:
            raise ValueError(f"Invalid Pauli label: {pl}")

        is_zero = rng.random() < p0
        outcome_bits.append('0' if is_zero else '1')

        # --- collapse & renormalise |ψ⟩ ------------------------------------
        if pl == 'Z':
            if is_zero:
                psi1[...] = 0
                psi /= np.sqrt(p0)
            else:
                psi0[...] = 0
                psi /= np.sqrt(1.0 - p0)

        else:  # X or Y
            if pl == 'X':
                plus  = (psi0 + psi1) / sqrt2
                minus = (psi0 - psi1) / sqrt2
                phase = 1.0
            else:  # Y
                plus  = (psi0 + 1j * psi1) / sqrt2
                minus = (psi0 - 1j * psi1) / sqrt2
                phase = 1j

            if is_zero:      # outcome ‘0’ (+1 eigenvalue)
                norm = np.sqrt(p0)
                psi0[...] = plus / norm
                psi1[...] =  plus / norm if pl == 'X' else -phase * plus / norm
            else:            # outcome ‘1’ (−1 eigenvalue)
                norm = np.sqrt(1.0 - p0)
                psi0[...] =  minus / norm
                psi1[...] = -minus / norm if pl == 'X' else  phase * minus / norm

    return ''.join(outcome_bits)


In [6]:
num_qubits = 20
state_dim = 1 << num_qubits
rng = np.random.default_rng(0)

# random normalised state
psi = rng.normal(size=state_dim) + 1j * rng.normal(size=state_dim)
psi /= np.linalg.norm(psi)

pauli_dirs = rng.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)
result = sample_state_fast(psi, pauli_dirs, rng=rng)
print('Measurement basis:', ''.join(pauli_dirs))
print('Outcome bits     :', result)


Measurement basis: YIXXYYYIYXIZXYXIXXYI
Outcome bits     : 00000100100000000000
